# Recolección de datos

In [1]:
import cv2 as cv
import csv
import copy
import itertools
import numpy as np
import mediapipe as mp
import os.path 
from os import walk

In [2]:
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.2,
    min_tracking_confidence=0.7,
)
os.chdir("../data/GESTURES_VIDEO")

## 2D

In [16]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [17]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [20]:
def logging_csv(dirname,name, label, clase, landmark_list):
    csv_path = f'../2d_DATA/{dirname}/{name}_{clase}.csv'
    if os.path.isfile(csv_path):
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([clase, label, *landmark_list]) 
    else:
        with open(csv_path, mode='w') as f:
            writer = csv.writer(f)
            columnas = ["class", "hand"]+[f'k{i}{axis}' for i in range(21) for axis in ("x", "y")]
            writer.writerow(columnas)
            writer.writerow([clase, label, *landmark_list])
    return

In [21]:
actualPath = os.getcwd()
listdir = os.listdir(actualPath)
w = walk(actualPath)
for (dirpath, dirnames, filenames), i in zip(w, range(0, (len(listdir) + 1))):
    for filename in filenames:
        vid = cv.VideoCapture(dirpath +"/"+filename)
        capturar = False
        while True:
            ret, frame = vid.read()
            if not ret:
                break
            key = cv.waitKey(10)
            if key == 27: #ESC
                break
            frame = cv.flip(frame, 1)
            result = hands.process(frame)
            if result.multi_hand_landmarks is not None:
                for hand_landmarks, handeness in zip(result.multi_hand_landmarks, result.multi_handedness):
                    landmark_list = calc_landmark_list(frame, hand_landmarks)
                    pre_processed_landmark_list = pre_process_landmark(landmark_list)
                    logging_csv(listdir[i-1],filename[:-4], handeness.classification[0].label, i-1, pre_processed_landmark_list)

## 3D

In [3]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y, landmark_z])

    return landmark_point

In [4]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y, base_z = 0, 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y, base_z = landmark_point[0], landmark_point[1], landmark_point[2] 

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y
        temp_landmark_list[index][2] = temp_landmark_list[index][2] - base_z

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [14]:
def logging_csv(dirname,name, label, clase, landmark_list):
    csv_path = f'../3d_DATA/{dirname}/{name}_{clase}.csv'
    if os.path.isfile(csv_path):
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([clase, label, *landmark_list]) 
    else:
        with open(csv_path, mode='w') as f:
            writer = csv.writer(f)
            columnas = ["class", "hand"]+[f'k{i}{axis}' for i in range(21) for axis in ("x", "y", "z")]
            writer.writerow(columnas)
            writer.writerow([clase, label, *landmark_list])
    return

In [15]:
actualPath = os.getcwd()
listdir = os.listdir(actualPath)
w = walk(actualPath)
for (dirpath, dirnames, filenames), i in zip(w, range(0, (len(listdir) + 1))):
    for filename in filenames:
        vid = cv.VideoCapture(dirpath +"/"+filename)
        capturar = False
        while True:
            ret, frame = vid.read()
            if not ret:
                break
            key = cv.waitKey(10)
            if key == 27: #ESC
                break
            frame = cv.flip(frame, 1)
            result = hands.process(frame)
            if result.multi_hand_landmarks is not None:
                for hand_landmarks, handeness in zip(result.multi_hand_landmarks, result.multi_handedness):
                    landmark_list = calc_landmark_list(frame, hand_landmarks)
                    pre_processed_landmark_list = pre_process_landmark(landmark_list)
                    logging_csv(listdir[i-1],filename[:-4], handeness.classification[0].label, i-1, pre_processed_landmark_list)